In [17]:
import os
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import pandas as pd

%matplotlib inline
# %matplotlib ipympl

In [21]:
# User input
stn_id = "196.35.0"
# stn_id = "18.4.0"

# Parameters to download
par_ids = [1001]  # Daily mean discharge

# Resolution of data to download
## N.B. can't get the hourly resolution data download to work, so have done that manually via Sildre for now
resolution = 'daily'
resolution_dict = {'daily': 1440,
                   'hourly': 60,
                   'instantaneous': 0}

# Start and end dates for download
st_dt = "1900-01-01"
end_dt = "2022-01-01"

# Where to save data
download_date = '2022-10'  # (for file naming): current month and year
# out_fpath = '../data/discharge/Q_målselvfossen_alldata_%s-resolution_downloaded-%s.csv' % (resolution, download_date)
out_fpath = '../data/discharge/Q_Storelva_alldata_%s-resolution_downloaded-%s.csv' % (resolution, download_date)

In [22]:
# Get daily mean discharge data
df = nivapy.da.query_nve_hydapi([stn_id], par_ids, st_dt, end_dt, resolution=1440)
df.head()

,station_id,station_name,parameter,parameter_name,parameter_name_eng,method,datetime,value,unit,correction,quality
0,196.35.0,Målselvfossen,1001,Vannføring,Discharge,Mean,1965-03-23 11:00:00+00:00,17.53410,m³/s,1,3
1,196.35.0,Målselvfossen,1001,Vannføring,Discharge,Mean,1965-03-24 11:00:00+00:00,17.09777,m³/s,1,3
2,196.35.0,Målselvfossen,1001,Vannføring,Discharge,Mean,1965-03-25 11:00:00+00:00,16.66732,m³/s,1,3
3,196.35.0,Målselvfossen,1001,Vannføring,Discharge,Mean,1965-03-26 11:00:00+00:00,16.38360,m³/s,2,3
4,196.35.0,Målselvfossen,1001,Vannføring,Discharge,Mean,1965-03-27 11:00:00+00:00,16.10248,m³/s,2,3


See here for an interpretation of the correction and quality integer values: https://hydapi.nve.no/UserDocumentation/

Quality types:
* Unknown = 0
* Uncontrolled = 1
* PrimaryControlled = 2
* SecondaryControlled = 3

Correction type indicates whether the value has been corrected. e.g.:
* 0: No changes
* 1: Manual- or ice correction
* 2: Interpolation
* 3: Value computed with models or/and other series
* 4: Daily mean computed with arithmetic mean. (Only used on series where we normally should use discharge curve)
* 13: The value is calculated based on a similar station or a station close by
* 14: The value is missing, and is only calculated based on statistical information for the data series

In [ ]:
# Reset index
df.set_index('datetime', inplace=True)

# Quick plot
ax = plt.figure(figsize=(12, 6)).add_subplot(111)
station_name = df['station_name'][0]
df['value'].plot(ax=ax)
plt.grid(True)
plt.xlabel('')
plt.suptitle('%s, %s' % (station_name, stn_id))
# plt.savefig(r'../../data/NVE/raw_data_plots/%s-%s.png' % (station_name, stn_id))

In [ ]:
yr_list = list(range(2014, 2022))

fig = plt.figure(figsize=(12, 25))
for i, year in enumerate(yr_list):
    ax = fig.add_subplot(len(yr_list), 1, i+1)
    df['value'].loc['%s-01-01' % year: '%s-12-31' % year].plot(ax=ax)
    plt.grid(True)
    plt.xlabel('')
    if year != 2021:
        frame1 = plt.gca()
        frame1.axes.xaxis.set_ticklabels([])
    plt.ylabel("Q, %s" % year)

plt.tight_layout()
# plt.savefig(r'../../data/NVE/raw_data_plots/yr_by_yr/%s-%s_yr-by-yr.png' % (station_name, stn_id))
# plt.close()

In [ ]:
# Save to csv
df.to_csv(out_fpath)